In [3]:
import tensorflow as tf

In [16]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_train[0])
x_train, x_test = x_train / 255.0, x_test / 255.0

(60000, 28, 28)
(60000,)
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 2

In [17]:

class Embedder(tf.keras.layers.Layer):
    def __init__(self, d_model, vocab):
        super(Embedder, self).__init__()
        self.emb = tf.keras.layers.Embedding(vocab, d_model)
        self.d_model = d_model
        self.vocab = vocab

    def call(self, x):

        max_len = x.get_shape()[1]
        print('max_len:', max_len)

        # shape == (batch_size, max_len, d_model)
        x = self.emb(x) * tf.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.get_positional_encoding(max_len)
        return x 

    def get_positional_encoding(self, max_len):
        """PE_(pos, 2i) = sin(pos/10000^(2i/d_model))
        PE_(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        """

        pos = np.expand_dims(np.arange(0, max_len), axis=1)
        div_term = np.array([[1 / np.power(10000, (2 * (i//2) / self.d_model)) for i in range(self.d_model)]])       

        pos = pos * div_term

        pe = np.zeros((max_len, self.d_model))
        pe[:, 0:self.d_model//2] = np.sin(pos[:, 0::2])
        pe[:, self.d_model//2:] = np.cos(pos[:, 0::2])

        pe = np.expand_dims(pe, 0)

        print(pe.shape)

        return tf.cast(pe, dtype=tf.float32) 


